In [1]:
# 数据操作
'''
    特殊的数据类型：
    1. 数组（Array）
    2. 动态变量（Variable）
    这两种数据类型可以在多种设备（CPU、GPU、TPU）上进行计算
    并支持自动求梯度的功能
'''

'\n    特殊的数据类型：\n    1. 数组（Array）\n    2. 动态变量（Variable）\n    这两种数据类型可以在多种设备（CPU、GPU、TPU）上进行计算\n    并支持自动求梯度的功能\n'

In [2]:
# 导库与环境设置
import numpy as np
import brainpy as bp
import jax.numpy as jnp
import brainpy.math as bm
import matplotlib.pyplot as plt

bm.set_platform('cpu')  # 新数据存储在内存中，使用CPU计算（在有相关资源时，也可以设置为GPU或TPU）

In [3]:
bm_array = bm.array([0, 1, 2, 3, 4, 5])     # 使用BrainPy创建数组
np_array = np.array([0, 1, 2, 3, 4, 5])     # 使用NumPy创建数组

print('bm_array: {0}'.format(bm_array))
print('np_array:'.format(np_array))

bm_array: [0 1 2 3 4 5]
np_array:


In [4]:
# bm.array()的四个参数
'''
    1. .ndim: 数组的轴数（维度）
    2. .shape: 数组的形状，用int32类型的元组表示每个维度的数组的大小，对n行m列的矩阵，形状就是(n, m)
    3. .size: 数组的元素数，等于各元素的积
    4. .dtype: 描述数组中元素类型的对象，可以使用标准的Python类型创建或指定元素类型
'''

'\n    1. .ndim: 数组的轴数（维度）\n    2. .shape: 数组的形状，用int32类型的元组表示每个维度的数组的大小，对n行m列的矩阵，形状就是(n, m)\n    3. .size: 数组的元素数，等于各元素的积\n    4. .dtype: 描述数组中元素类型的对象，可以使用标准的Python类型创建或指定元素类型\n'

In [5]:
# 例子
t1 = bm.array([[[0, 1, 2, 3], [1, 2, 3, 4], [4, 5, 6, 7]],
               [[0, 0, 0, 0], [-1, 1, -1, 1], [2, -2, 2, -2]]])
print('t1.ndim:{0}'.format(t1.ndim))            # 轴数
print('t1.shape:{0}'.format(t1.shape))          # 形状
print('t1.size:{0}'.format(t1.size))            # 元素积
print('t1.dytpe:{0}'.format(t1.dtype))          # 数据类型

t1.ndim:3
t1.shape:(2, 3, 4)
t1.size:24
t1.dytpe:int32


In [6]:
# 获取t1的值
t1_value = t1.value
print("Value of t1:\n {0}".format(t1_value))

Value of t1:
 [[[ 0  1  2  3]
  [ 1  2  3  4]
  [ 4  5  6  7]]

 [[ 0  0  0  0]
  [-1  1 -1  1]
  [ 2 -2  2 -2]]]


In [7]:
# 动态变量（Variable）：随时间动态变化的数组

In [8]:
# 要将数组转换为动态变量，只需要将数组包装到brainpy.math.Variable中
t = bm.arange(4)
v = bm.Variable(t)
print('Dynamic variable: {0}'.format(v))                    # 获取动态变量
print('Value of dynamic variable: {0}'.format(v.value))     # 获取动态变量的值

Dynamic variable: [0 1 2 3]
Value of dynamic variable: [0 1 2 3]


In [9]:
# 动态变量的更新
v = bm.Variable(bm.arange(4))
print('Before: {0}'.format(v))
v = v + 2           # 开辟一个新空间来储存新结果，原来的动态变量没有发生改变（类似指针）
print('After: {0}'.format(v))
# 使用In-Place Updating修改动态变量内部的值

Before: [0 1 2 3]
After: [2 3 4 5]


In [10]:
# 1. 索引
v = bm.Variable(bm.arange(4))
print('Before: {0}'.format(v))
v[0] = 10
print('After: {0}'.format(v))

Before: [0 1 2 3]
After: [10  1  2  3]


In [11]:
# 2. 切片
v = bm.Variable(bm.arange(4))
print('Before: {0}'.format(v))
v[1:3] = 9
print('After: {0}'.format(v))

Before: [0 1 2 3]
After: [0 9 9 3]


In [12]:
# 3. 增量赋值
v = bm.Variable(bm.arange(4))
print('Before: {0}'.format(v))
v += 1
print('After: {0}'.format(v))

Before: [0 1 2 3]
After: [1 2 3 4]


In [13]:
# 4. .value赋值
v.value = bm.arange(4)
print('Value of v: {0}'.format(v))

Value of v: [0 1 2 3]


In [14]:
'''
    注意：
    在使用新数组覆盖动态变量的数据时，
    一定要保证数组的形状、元素类型与动态变量完全一致，
    否则会报错
'''

# 例子
try:
    v.value = bm.array([1., 1., 1., 0.])        # 将float32类型数组赋给int32类型动态变量
except Exception as e:
    print(type(e), e)

<class 'brainpy.errors.MathError'> The dtype of the original data is int32, while we got float32.


In [15]:
# 5. .update方法（与.value类似）
v.update(bm.array([3, 4, 5, 6]))
print('Value of v: {0}'.format(v))

Value of v: [3 4 5 6]


In [16]:
# 批处理大小(Batch Size)：每次传给网络的样本数量
# 在机器学习任务重（训练网络），用户在初始化一个动态变量时，需要声明批处理维度
# 参数batch_axis表示数组中的批处理维度
dyn_var = bm.Variable(bm.zeros((1, 100)), batch_axis=0)
print('dyn_var.shape: {0}'.format(dyn_var.shape))

dyn_var = bm.ones((10, 100))    # 批处理大小变为10

dyn_var.shape: (1, 100)
